In [ ]:
### Example of a 1D Channel driven flow

Model Setup
-----------

2D, Stokes Equation with noslip BC at top and bottom boundary. Pressure gradient across horizonal axis drives flow

\\[
\frac{\partial \tau}{\partial y} = \mu \frac{\partial^{2} \mathbf{u}}{\partial{y}^{2}} = - \frac{\partial p}{\partial x}
\\]

\\[
\nabla \cdot \mathbf{u} = 0
\\]

with $x = [0,1],  y =[0,h]$

$\mathbf{u}(x,y=1) = \mathbf{u}(x,y=0) = 0$

------

Effectively a 1D problem in $y$-axis, described by the analytic function

$ \mathbf{u}(y) = - \left[ \frac{1}{2 \mu} \frac{\partial p }{\partial x} ( y^{2} - h y ), 0 \right]$

In [ ]:
import underworld as uw
from underworld import function as fn
import glucifer
import math
import numpy as np

Setup parameters
-----

In [ ]:
# Set domain dimension & size
dim = 2
h = 2.0
boxLength = 1.0
# Set grid resolution.
res = 20

# Set a constant viscosity.
viscosity = 1.


In [ ]:
mesh = uw.mesh.FeMesh_Cartesian( elementType = ("Q1/dQ0"), 
                                 elementRes  = (res, res), 
                                 minCoord    = (-1., 0.), 
                                 maxCoord    = (boxLength, h),
                                 periodic = (False,False))

velocityField   = uw.mesh.MeshVariable( mesh=mesh,         nodeDofCount=dim )
pressureField   = uw.mesh.MeshVariable( mesh=mesh.subMesh, nodeDofCount=1 )

# initialise velocity, pressure and temperatureDot field
velocityField.data[:] = [0.,0.]
pressureField.data[:] = 0.

In [ ]:
jWalls = mesh.specialSets["MinJ_VertexSet"] + mesh.specialSets["MaxJ_VertexSet"]
iWalls = mesh.specialSets["MinI_VertexSet"] + mesh.specialSets["MaxI_VertexSet"]
allWalls = iWalls + jWalls

In [ ]:
vBC = uw.conditions.DirichletCondition( variable = velocityField, 
                                        indexSetsPerDof = (jWalls, allWalls) )

# constant stress definition
dp_dx = 5.0
xcoord = fn.coord()[0]
stress = xcoord*(dp_dx,dp_dx,0.0) 

# This stress is multiplied by the wall normal to produce a surface force, via divergence-theorem
# The left wall normal unit vector is (-1,0)
# The right wall normal unit vector is (1,0)
# stress_i_j * normal_j = surface_force

nbc = uw.conditions.NeumannCondition( stress, 
                                      variable=velocityField,
                                      nodeIndexSet=(iWalls) )

In [ ]:
stokesPIC = uw.systems.Stokes( velocityField = velocityField, 
                               pressureField = pressureField,
                               conditions    = [vBC, nbc],
                               fn_viscosity   = viscosity, 
                               fn_bodyforce   = 0.0 )

solver = uw.systems.Solver( stokesPIC )
solver.solve()

In [ ]:
fig = glucifer.Figure()
velmagfield = uw.function.math.sqrt( uw.function.math.dot( velocityField, velocityField ) )
fig.append( glucifer.objects.VectorArrows(mesh, velocityField, arrowHead=0.2, scaling=0.1) )
fig.append( glucifer.objects.Mesh(mesh) )
fig.append( glucifer.objects.Surface( mesh, pressureField ) )
fig.show()

In [ ]:
ids = mesh.specialSets["MaxI_VertexSet"]
coords = mesh.data[ids.data] # xcoords
V = velocityField.evaluate(ids)
gradV = velocityField.fn_gradient.evaluate(ids)

u = V[:,0] ; v = V[:,1]
du_dx = gradV[:,0] ; du_dy = gradV[:,1]
dv_dx = gradV[:,2] ; dv_dy = gradV[:,3]

strainRate = fn.tensor.symmetric( velocityField.fn_gradient )
devstress = 2.0 * strainRate

In [ ]:
def exact(y):
    ana_u = -1.0 / (2.0 * viscosity)* dp_dx * (y**2 - h*y)
    return ana_u

In [ ]:
ana_u = exact(coords[:,1])

abserr = uw.utils.nps_2norm(ana_u - u)
mag = uw.utils.nps_2norm(ana_u)
relerr = abserr / mag

from mpi4py import MPI
comm = MPI.COMM_WORLD

if uw.rank() == 0:
    print "Relative error against analytic solution:", relerr
    if relerr > 1.0e-6:
        raise RuntimeError("The numerical solution is outside the error threshold of the analytic solution." \
                           "The Relative error was ", relerr," the threshold is ", threshold)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as pyplot
import matplotlib.pylab as pylab
pylab.rcParams[ 'figure.figsize'] = 12, 6
pyplot.plot(coords[:,1], u, 'o', color = 'green', label='vx')
pyplot.plot(coords[:,1], v, 'o', color = 'red', label='vy')
big = np.linspace(0.0,h)
pyplot.plot(big, exact(big), 'D', color = 'purple', label='exact_vx', markersize=2)
pyplot.legend()
pyplot.xlabel('y coords at x=xmax')
pyplot.ylabel('velocity')
pyplot.show()

In [ ]:
pyplot.plot(coords[:,1], du_dy, 'o', color = 'purple', label='du_dy')
pyplot.plot(coords[:,1], du_dx, 'o', color = 'black', label='du_dx')
pyplot.plot(coords[:,1], dv_dy, 'o', color = 'orange', label='dv_dy')
pyplot.plot(coords[:,1], dv_dx, 'o', color = 'red', label='dv_dx')
pyplot.legend()
pyplot.xlabel('y coords at x=xmax')
pyplot.ylabel('velocity gradients')
pyplot.show()

In [ ]:
pyplot.plot(coords[:,1], strainRate.evaluate(ids)[:,2], 'o', color = 'purple', label='e_shear')
pyplot.plot(coords[:,1], devstress.evaluate(ids)[:,2], 'o', label='tau_shear')
pyplot.legend()
pyplot.xlabel('y coords at x=xmax')
pyplot.ylabel('strain rate')
pyplot.show()